In [1]:
!git clone https://github.com/ai4se-course/ai4se-hse-course-24-25.git

Cloning into 'ai4se-hse-course-24-25'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 22 (delta 3), reused 1 (delta 1), pack-reused 7 (from 1)
Receiving objects: 100% (22/22), 9.56 KiB | 181.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.


Установите и настройте библиотеку tree-sitter для синтаксического анализа кода на Python и необходимые файлы для языка (пакеты tree-sitter и tree-sitter-python).

In [2]:
!pip install -r "ai4se-hse-course-24-25/02-func-name-suggestion/requirements.txt"
!pip install -r "ai4se-hse-course-24-25/02-func-name-suggestion/requirements_dev.txt"

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.6/542.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=8f0797fd0e7037cd1362c4c3100ba63d76ef9c820b856dd7301441a407c007ee
  Store

Загрузите данные из CodeSearchNet и проанализируйте их структуру (используйте load_dataset из библиотеки datasets из экосистемы HuggingFace, потребуется использовать флаг trust_remote_code=True, не забудьте выбрать язык python, в качестве аргумента splitможно сразу передать test). В процессе выполнения задания можно ограничить количество примеров, например, до первой 1000.

In [3]:
from datasets import load_dataset
ds = load_dataset('code-search-net/code_search_net', name="python", split='test[:1000]', trust_remote_code=True)
ds


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 1000
})

С помощью библиотеки tree-sitter выполните синтаксический разбор полей whole_func_string и реализуйте извлечения следующих элементов из построенных деревьев:

- имя функции
- тело функции без комментариев и документации
- тело функции с комментариями с документацией (docstrings)

Реализация может быть выполнена, с помощью обходчика синтаксического дерева (AST-Visitor) или с использованием query API библиотеки tree-sitter


In [4]:
import tree_sitter_python as tspython
from tree_sitter import Language, Parser, Tree, Node
from typing import Generator
from datasets import Dataset

In [5]:
def traverse_tree(tree: Tree) -> Generator[Node, None, None]:
    cursor = tree.walk()

    visited_children = False
    while True:
        if not visited_children:
            if cursor.node.type == 'comment':
                yield [cursor.node.start_byte, cursor.node.end_byte + 1]
            if not cursor.goto_first_child():
                visited_children = True
        elif cursor.goto_next_sibling():
            visited_children = False
        elif not cursor.goto_parent():
            break

In [6]:
def get_function_body_node(stb):
    PY_LANGUAGE = Language(tspython.language())
    parser = Parser(PY_LANGUAGE)
    tree = parser.parse(stb)

    root_node = tree.root_node
    assert root_node.type == 'module'

    function_node = root_node.children[0]
    assert function_node.type == 'function_definition'

    function_name_node = function_node.children[1]
    function_name = str(stb[function_name_node.start_byte:function_name_node.end_byte], "utf8")

    function_body_node = function_node.child_by_field_name("body")

    function_parameters_node = function_node.child_by_field_name("parameters")
    function_arguments = str(stb[function_parameters_node.start_byte:function_parameters_node.end_byte], "utf8")

    doc_string = ""
    for i in range(len(function_body_node.children)):
      if function_body_node.children[i].type == "expression_statement" and function_body_node.children[i].children[0].type == "string":
        end_byte = function_body_node.children[i].children[0].children[1].end_byte
        while end_byte > 0 and stb[end_byte-1] == 32 or stb[end_byte-1] == 10:
          end_byte = end_byte-1
        start_byte = function_body_node.children[i].children[0].children[1].start_byte
        while end_byte > start_byte and stb[start_byte] == 32 or stb[start_byte] == 10:
          start_byte = start_byte+1

        doc_string = str(stb[start_byte:end_byte], "utf8")
        break

    return function_name, function_arguments, doc_string, function_body_node

In [7]:
def get_function_body_without_comments(stb, function_body_node):
    node_names = [function_body_node.start_byte]
    for i in range(len(function_body_node.children)):
      if function_body_node.children[i].type == "expression_statement" and function_body_node.children[i].children[0].type == "string":
        node_names = [function_body_node.children[i].end_byte + 1]
        break

    node_names = node_names + [x for p in list(map(lambda node: node, traverse_tree(function_body_node))) for x in p] + [function_body_node.end_byte]
    result = ""
    for i in range(0, len(node_names), 2):
      result = result + str(stb[node_names[i]:node_names[i + 1]], "utf8") + "\n"

    result = "\n".join([line for line in result.split('\n') if line.strip()])
    return result

In [8]:
def get_function_body_with_comments(stb, function_body_node):
    return str(stb[function_body_node.start_byte:function_body_node.end_byte], "utf8")

Приведите примеры извлеченных функций




In [9]:
st = 'def sina_xml_to_url_list(xml_data):\n    """str->list\n    Convert XML to URL List.\n    From Biligrab.\n    """\n    rawurl = []\n    # Comment1\n    # Comment 2\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName(\'durl\'):\n        url = node.getElementsByTagName(\'url\')[0]  # Comment 3\n        rawurl.append(url.childNodes[0].data)\n    return rawurl'
stb = bytes(st, "utf8")
st

'def sina_xml_to_url_list(xml_data):\n    """str->list\n    Convert XML to URL List.\n    From Biligrab.\n    """\n    rawurl = []\n    # Comment1\n    # Comment 2\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName(\'durl\'):\n        url = node.getElementsByTagName(\'url\')[0]  # Comment 3\n        rawurl.append(url.childNodes[0].data)\n    return rawurl'

In [10]:
function_name, function_arguments, doc_string, function_body_node = get_function_body_node(stb)
function_name, function_arguments, doc_string

('sina_xml_to_url_list',
 '(xml_data)',
 'str->list\n    Convert XML to URL List.\n    From Biligrab.')

In [11]:
get_function_body_with_comments(stb, function_body_node)

'"""str->list\n    Convert XML to URL List.\n    From Biligrab.\n    """\n    rawurl = []\n    # Comment1\n    # Comment 2\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName(\'durl\'):\n        url = node.getElementsByTagName(\'url\')[0]  # Comment 3\n        rawurl.append(url.childNodes[0].data)\n    return rawurl'

In [12]:
get_function_body_without_comments(stb, function_body_node)

"    rawurl = []\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName('durl'):\n        url = node.getElementsByTagName('url')[0]  \n        rawurl.append(url.childNodes[0].data)\n    return rawurl"

Приведите примеры извлеченных функций




In [13]:
st = ds["whole_func_string"][0]
stb = bytes(st, "utf8")
st

'def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')'

In [14]:
function_name, function_arguments, doc_string, function_body_node = get_function_body_node(stb)
function_name, function_arguments, doc_string

('get_vid_from_url', '(url)', 'Extracts video ID from URL.')

In [15]:
get_function_body_with_comments(stb, function_body_node)

'"""Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')'

In [16]:
get_function_body_without_comments(stb, function_body_node)

"        return match1(url, r'youtu\\.be/([^?/]+)') or \\\n          match1(url, r'youtube\\.com/embed/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/v/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/watch/([^/?]+)') or \\\n          parse_query_param(url, 'v') or \\\n          parse_query_param(parse_query_param(url, 'u'), 'v')"

Для проверки корректности извлечения сравните извлеченные имена с полем `func_name`, документацию с полем `func_documentation_string` в исходном датасете. Имейте ввиду, что оригинальное поле `func_name` содержит полное квалифицированное имя `(ClassA.foo)`, в задании этого не требуется, можно ограничиться просто именем `(foo)`.

In [17]:
for i in range(100):
  function_name, _, _, _ = get_function_body_node(bytes(ds["whole_func_string"][i], "utf8"))
  l = ds["func_name"][i].split('.')
  if len(l) > 1 :
    a, b = l
    assert b == function_name
  else:
    assert ds["func_name"][i] == function_name

In [18]:
for i in range(100):
  _, _, doc_string, _ = get_function_body_node(bytes(ds["whole_func_string"][i], "utf8"))
  assert ds["func_documentation_string"][i] == doc_string

**Использование предобученных моделей только на исходном коде**

Загрузите предобученную модель (например, CodeT5+)

In [19]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

checkpoint = "Salesforce/codet5p-220m"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

Используйте подготовленный на первом этапе датасет, содержащий только тела функций без каких либо комментариев и их имена, для предсказания имен функций.


In [20]:
def get_function_name_and_body_without_comments(i, stb):
  function_name, function_arguments, doc_string, function_body_node = get_function_body_node(stb)
  function_body_without_comments = get_function_body_without_comments(stb, function_body_node)
  return function_name, f"def <extra_id_0>" + function_arguments + ":\n" + function_body_without_comments

In [21]:
def gen():
    for i in range(ds.num_rows):
        function_name, function_body_without_comments = get_function_name_and_body_without_comments(i, bytes(ds["whole_func_string"][i], "utf8"))
        yield {"function_body_without_comments": function_body_without_comments, "function_name": function_name}
ds1 = Dataset.from_generator(gen)
ds1[0]

Generating train split: 0 examples [00:00, ? examples/s]

{'function_body_without_comments': "def <extra_id_0>(url):\n        return match1(url, r'youtu\\.be/([^?/]+)') or \\\n          match1(url, r'youtube\\.com/embed/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/v/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/watch/([^/?]+)') or \\\n          parse_query_param(url, 'v') or \\\n          parse_query_param(parse_query_param(url, 'u'), 'v')",
 'function_name': 'get_vid_from_url'}

In [22]:
inputs = tokenizer(ds1["function_body_without_comments"], truncation=True, padding=True, max_length=80, return_tensors="pt").to(device)

In [23]:
outputs = model.generate(**inputs, max_length=80)

In [24]:
predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
predictions = list(map(lambda s: s.split(' ')[:2][-1], predictions))

Оцените результаты работы моделей без тонкой настройки, используя метрики качества Exact Match (полное совпадение) и ROUGE-score (можно использовать реализацию из библиотеки `evaluate` `evaluate.load('exact_match')`, `evaluate.load('rouge')`. Показатель EM на первой 1000 примеров тестовой выборки должен быть в окрестности 0.145; ROUGE-1 в окрестности 0.38.

In [25]:
from evaluate import load

exact_match = load("exact_match")
results = exact_match.compute(references=ds1["function_name"], predictions=predictions, ignore_case=True, ignore_punctuation=True, ignore_numbers=True)
results

{'exact_match': 0.157}

In [26]:
bad_exact = []
for i in range(ds1.num_rows):
  if exact_match.compute(references=[ds1["function_name"][i]], predictions=[predictions[i]])["exact_match"] == 0:
      bad_exact.append(i)
  if len(bad_exact) > 10 :
    break
bad_exact

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]

In [27]:
rouge_match = load("rouge")
results = rouge_match.compute(references=ds1["function_name"], predictions=predictions)
results

{'rouge1': 0.3459225274725278,
 'rouge2': 0.16446904761904768,
 'rougeL': 0.34539618437118436,
 'rougeLsum': 0.34510442612942616}

In [28]:
bad_rouge1 = []
for i in range(ds1.num_rows):
  if rouge_match.compute(references=[ds1["function_name"][i]], predictions=[predictions[i]])["rouge1"] == 0:
      bad_rouge1.append(i)
  if len(bad_rouge1) > 10 :
    break
bad_rouge1

[0, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13]

Посмотрим на случайные худшие примеры по метрикам
`[0, 2, 7, 10, 11]`

In [32]:
for i in [0, 2, 7, 10, 11]:
  print(i, ds1["function_name"][i], predictions[i], "\n---------------------------------------\n", ds1["function_body_without_comments"][i])

0 get_vid_from_url match1 
---------------------------------------
 def <extra_id_0>(url):
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')
2 makeMimi md5 
---------------------------------------
 def <extra_id_0>(upid):
    strSeed = "gGddgPfeaf_gzyr"
    prehash = upid + "_" + strSeed
    return md5(prehash.encode('utf-8')).hexdigest()
7 ucas_download_playlist get_playtitle 
---------------------------------------
 def <extra_id_0>(url, output_dir = '.', merge = False, info_only = False, **kwargs):
    html = get_content(url)
    parts = re.findall( r'(getplaytitle.do\?.+)"', html)
    assert parts, 'No part found!'
    for part_path in parts:
        ucas_download('http://v.ucas.ac.cn/course/' + part_pat

0 - модель нашла что-то вообще странное \\
2 - без комментариев текст функции вообще не связан с названием \\
7 - похоже, метрики не отражают, что не все так плохо \\
10 - предложенное решение похоже по смыслу, но не отражает смысл функции, не совсем плохо, но есть сильно куда улучшать \\
11 - модель нашла что-то вообще странное


**Использование предобученных моделей на исходном коде, документации и комментариях**

Загрузите предобученную модель (например, CodeT5+)



In [33]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

checkpoint = "Salesforce/codet5p-220m"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint).to(device)

Используйте подготовленный на первом этапе датасет, содержащий тела функций, их имена, документацию и комментарии.



In [34]:
def get_function_name_and_body_with_comments(i, stb):
  function_name, function_arguments, doc_string, function_body_node = get_function_body_node(stb)
  function_body_with_comments = get_function_body_with_comments(stb, function_body_node)
  return function_name, f"def <extra_id_0>" + function_arguments + ":\n" + function_body_with_comments

In [35]:
get_function_name_and_body_with_comments(0, bytes(ds["whole_func_string"][0], "utf8"))

('get_vid_from_url',
 'def <extra_id_0>(url):\n"""Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')')

In [36]:
from datasets import Dataset
def gen():
    for i in range(ds.num_rows):
        function_name, function_body_with_comments = get_function_name_and_body_with_comments(i, bytes(ds["whole_func_string"][i], "utf8"))
        yield {"function_body_with_comments": function_body_with_comments, "function_name": function_name}
ds2 = Dataset.from_generator(gen)
ds2[0]

Generating train split: 0 examples [00:00, ? examples/s]

{'function_body_with_comments': 'def <extra_id_0>(url):\n"""Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')',
 'function_name': 'get_vid_from_url'}

In [37]:
inputs = tokenizer(ds2["function_body_with_comments"], truncation=True, padding=True, max_length=80, return_tensors="pt").to(device)

In [38]:
outputs = model.generate(**inputs, max_length=80)
tokenizer.decode(outputs[5], skip_special_tokens=True)

' dictify dictify_xml dictify dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify_xml dictify'

In [39]:
predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
predictions = list(map(lambda s: s.split(' ')[:2][-1], predictions))

Оцените результаты работы моделей без тонкой настройки на таких расширенных примерах с использованием тех же метрик. Показатель EM на первой 1000 примеров тестовой выборки должен быть в окрестности 0.209; ROUGE-1 в окрестности 0.49.



In [40]:
exact_match = load("exact_match")
results = exact_match.compute(references=ds2["function_name"], predictions=predictions, ignore_case=True, ignore_punctuation=True, ignore_numbers=True)
results

{'exact_match': 0.215}

In [41]:
bad_exact = []
for i in range(ds2.num_rows):
  if exact_match.compute(references=[ds2["function_name"][i]], predictions=[predictions[i]])["exact_match"] == 0:
      bad_exact.append(i)
  if len(bad_exact) > 10 :
    break
bad_exact

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]

In [42]:
rouge_match = load("rouge")
results = rouge_match.compute(references=ds2["function_name"], predictions=predictions)
results

{'rouge1': 0.47189603174603223,
 'rouge2': 0.23468690476190468,
 'rougeL': 0.4689229076479078,
 'rougeLsum': 0.468839177489178}

In [43]:
bad_rouge1 = []
for i in range(ds2.num_rows):
  if rouge_match.compute(references=[ds2["function_name"][i]], predictions=[predictions[i]])["rouge1"] == 0:
      bad_rouge1.append(i)
  if len(bad_rouge1) > 10 :
    break
bad_rouge1

[0, 2, 3, 6, 7, 10, 11, 12, 15, 16, 18]

Посмотрим на случайные худшие примеры по метрикам
`[0, 2, 7, 11]`

In [44]:
for i in [0, 2, 7, 10, 11]:
  print(i, ds2["function_name"][i], predictions[i], "\n---------------------------------------\n", ds2["function_body_with_comments"][i])

0 get_vid_from_url extract_video_id 
---------------------------------------
 def <extra_id_0>(url):
"""Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')
2 makeMimi makeMimiLocal 
---------------------------------------
 def <extra_id_0>(upid):
"""From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js
    Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal
    L110"""
    strSeed = "gGddgPfeaf_gzyr"
    prehash = upid + "_" + strSeed
    return md5(prehash.encode('utf-8')).hexdigest()
7 ucas_download_playlist get_course_page 
---------------------------------------
 def <extra_id_0>(url, output_dir = '.', merge = False, info_only = False, **kwargs):
'''c

0 - модель нашла название получше чем было \\
2 - очень похоже, метрики не отражают \\
7 - предложенное решение похоже по смыслу, но по тексту совсем другое \\
10 - модель предложила не только имя функции, но и дописать перед функцией еще кучу кода, а мы делили по проблеам и взяли кусок этого кода. Не совсем то, чего хотелось бы (без комментариев тоже такое видела, но при финальной отчетности не смогла откопать пример) \\
11 - предложенное решение похоже по смыслу, но совсем не идеально и есть куда улучшать

**Анлогично первому практическому заданию подготовьте краткий отчет, который будет включать сравнение по метрикам Exact Match и ROUGE-score результатов работы с использованием документации и комментариев с результатами, полученными только на основе исходного кода. В отчет также включите анализ небольшого числа примеров с наихудшими значениями метрик.**

**Без комментов** \\
0 - модель нашла что-то вообще странное \\
2 - без комментариев текст функции вообще не связан с названием \\
7 - похоже, метрики не отражают, что не все так плохо \\
10 - предложенное решение похоже по смыслу, но не отражает смысл функции, не совсем плохо, но есть сильно куда улучшать \\
11 - модель нашла что-то вообще странное


**С комментами** \\
0 - модель нашла название получше чем было \\
2 - очень похоже, метрики не отражают \\
7 - предложенное решение похоже по смыслу, но по тексту совсем другое \\
10 - модель предложила не только имя функции, но и дописать перед функцией еще кучу кода, а мы делили по проблеам и взяли кусок этого кода. Не совсем то, чего хотелось бы (без комментариев тоже такое видела, но при финальной отчетности не смогла откопать пример) \\
11 - предложенное решение похоже по смыслу, но совсем не идеально и есть куда улучшать

**Без комментов** \\
```
{'exact_match': 0.157}

{'rouge1': 0.3459225274725278,
 'rouge2': 0.16446904761904768,
 'rougeL': 0.34539618437118436,
 'rougeLsum': 0.34510442612942616}
```

**С комментами** \\
```
{'exact_match': 0.215}

{'rouge1': 0.47189603174603223,
 'rouge2': 0.23468690476190468,
 'rougeL': 0.4689229076479078,
 'rougeLsum': 0.468839177489178}
```

Видим, что все метрики стали ожидаемо **лучше** **при добавлении комментов**. Кроме того, на случайно выбранных 5 примерах тоже видно улучшение. 0 и 11 из чего-то странного превратились во что-то осмысленное. 2 стало лучше. 7 ухудшился не критично.

**Сложности**

- Много пришлось повозиться с tree_sitter
- Помогите пожалуйста, я так больше не могу
- Низя запускать без доп параметров весь датасет